In [ ]:
# importing libraries
import os

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline
import time
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torchvision.utils import make_grid

# --- 

"""We'll download the images in PNG format from [this page](https://course.fast.ai/datasets), using some helper functions from the `torchvision` and `tarfile` packages.
#..............................our part...................................#

function  x + (L / (Q * (x + I) ** P + Z)) - (L / (Q * (x - I) ** P + Z))   https://www.desmos.com/calculator/0hiyjyun2f (move points to play with the parameters)

"""
#globals 
#----------------------------------------------------------------------------#
L=0
Q=0
P=0
Z=0
I=0
#----------------------------------------------------------------------------#


setAll=False# will train and predict useing all the data ignoring the 2 above
#if false set the data manulaiy 
#-----------------------------------------#
val_percent=0.5#x*5000 = dataset
train_percent=0.01#x*50000 = dataset

#old method:
# train_running_size=1000#train size of data {max is 50000}
# val_running_Size=1000#val size of data {max is 5000}

#-----------------------------------------#



#Relu parameters to compare
#-----------------------------------------#
addMaxRelu=True
addMean=True
HowManyTimesReRunRelu=10
#-----------------------------------------#




checkCertainParams=False#if the one above is true set the parameters u want to check, otherwise it will igonre the value:
#-----------------------------------------------------------------------------------------#
howManyTimeToCheckTheCertainParams=10# how many times to run the cetrtain params 
addMaxToCertain=True#will return the max score of the certain params
addMinToCertain=True#will return the min score of the certain params
addMeanToCertain=True#will return the mean of all the score's of the certain params
#-----------------------------------------------------------------------------------------#

#Layers const parameters for dynamic function, each number represent a diffrent layer:
#-----------------------------------------------------------------------------------------#
#layer 1 values 
L1=18.803015465431965
Q1=1.7782794100389232
P1=2
Z1=10
I1=1

#layer 2 values 
L2=7.071067811865475
Q2=1.7782794100389232
P2=P
Z2=Z
I2=1


#layer 3 values 
L3=13.572088082974531
Q3=0.1
P3=P
Z3=Z
I3=-1

#layer 4 values 
L4=3.6840314986403864
Q4=0.1
P4=P
Z4=Z
I4=-1.4422495703074083

#layer 5 values 
L5=3.6840314986403864
Q5=1
P5=P
Z5=Z
I5=2.080083823051904

#layer 6 values 
L6=2.6591479484724942
Q6=0.05623413251903491
P6=P
Z6=Z
I6=1.4422495703074083

#layer 7 values 
L7=7.071067811865475
Q7=0.05623413251903491
P7=P
Z7=Z
I7=1

#layer 8 values 
L8=0
Q8=0
P8=P
Z8=Z
I8=0
#-----------------------------------------------------------------------------------------#



#if checkCertainParams is false, config the range you'd like to check
#-----------------------------------------------------------------------------------------#
LogarithmINC=True#insted of incresing when checking parameters with a constant , will will get to the to value useing geometric progression
SIZEI=3#how many i to check
SIZEL=4#how many l to check
SIZEQ=4#how many q to check
#what number the paramter start and end
fromI=-1
toI=-3

fromL=1
toL=50

fromQ=0.01
toQ=10
#size of splits will, example fromI=1 toI=3,SIZEI=3 then i will check 1,2,3
#-----------------------------------------------------------------------------------------#



#Layers SetDynamic:# else the layers parameters for the function will be the one above (when l is 0 then dynamic = relu)
#-----------------------------------------------------------------------------------------#
DynamicLayer1=False
DynamicLayer2=False
DynamicLayer3=False
DynamicLayer4=False

DynamicLayer5=False
DynamicLayer6=False
DynamicLayer7=False
DynamicLayer8=False
#-----------------------------------------------------------------------------------------#


#------------------nn parameters----------------#
batch_size = 512 #hyper parameter
epochs = 32
random_seed = 1
opt_func = torch.optim.Adam

lr = 0.001#start learning rate
max_lr = 0.01# maximun learning rate
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam#optimizer
#----------------------------------------------#



#Don't change:
#-------------#
val_size = 5000
#-------------#

In [ ]:

def relu(input):
    x=input
    x[x<0]=0
    return x

class RELU(nn.Module):
    def __init__(self):
        super().__init__() # init the base class
    def forward(self, input):
        return relu(input) # simply apply already implemented SiLU

def dynamic(input):
    x=input
    x[x<0]=0

    if (checkCertainParams):
        x = x + (L0 / (Q0 * (x + I0) ** P0 + Z0)) - (L / (Q0 * (x - I0) ** P0 + Z0))
    else:
        x = x + (L / (Q * (x + I) ** P + Z)) - (L / (Q * (x - I) ** P + Z))
    return x


class Dynamic(nn.Module):
    def __init__(self):
        super().__init__() # init the base class
    def forward(self, input):
        return dynamic(input) # simply apply already implemented SiLU
activation_function=Dynamic()
def a_layer(input):
    x=input
    x[x<0]=0
    if DynamicLayer1:
      x = x + (L / (Q * (x + I) ** P + Z)) - (L / (Q * (x - I) ** P + Z))
    else:
      LIN=L1
      QIN=Q1
      IIN=I1
      PIN=P1
      ZIN=Z1
      x = x + (LIN / (QIN * (x + IIN) ** PIN + ZIN)) - (LIN / (QIN * (x - IIN) ** PIN + ZIN))

    return x

class A_layer_activation(nn.Module):
    def __init__(self):
        super().__init__() # init the base class
    def forward(self, input):
        return a_layer(input) # simply apply already implemented SiLU



def b_layer(input):
    x=input
    x[x<0]=0
    if DynamicLayer2:
      x = x + (L / (Q * (x + I) ** P + Z)) - (L / (Q * (x - I) ** P + Z))
    else:
      LIN=L2
      QIN=Q2
      IIN=I2
      PIN=P2
      ZIN=Z2
      x = x + (LIN / (QIN * (x + IIN) ** PIN + ZIN)) - (LIN / (QIN * (x - IIN) ** PIN + ZIN))

    return x

class B_layer_activation(nn.Module):
    def __init__(self):
        super().__init__() # init the base class
    def forward(self, input):
        return b_layer(input) # simply apply already implemented SiLU




def c_layer(input):
    x=input

    if DynamicLayer3:
      x = x + (L / (Q * (x + I) ** P + Z)) - (L / (Q * (x - I) ** P + Z))
    else:
      LIN=L3
      QIN=Q3
      IIN=I3
      PIN=P3
      ZIN=Z3
      x = x + (LIN / (QIN * (x + IIN) ** PIN + ZIN)) - (LIN / (QIN * (x - IIN) ** PIN + ZIN))
    return x

class C_layer_activation(nn.Module):
    def __init__(self):
        super().__init__() # init the base class
    def forward(self, input):
        return c_layer(input) # simply apply already implemented SiLU



def d_layer(input):
    x=input
    x[x<0]=0

    if DynamicLayer4:
      x = x + (L / (Q * (x + I) ** P + Z)) - (L / (Q * (x - I) ** P + Z))
    else:
      LIN=L4
      QIN=Q4
      IIN=I4
      PIN=P4
      ZIN=Z4
      x = x + (LIN / (QIN * (x + IIN) ** PIN + ZIN)) - (LIN / (QIN * (x - IIN) ** PIN + ZIN))
    return x

class D_layer_activation(nn.Module):
    def __init__(self):
        super().__init__() # init the base class
    def forward(self, input):
        return d_layer(input) # simply apply already implemented SiLU



def e_layer(input):
    x=input
    x[x<0]=0
    if DynamicLayer5:
      x = x + (L / (Q * (x + I) ** P + Z)) - (L / (Q * (x - I) ** P + Z))
    else:
      LIN=L5
      QIN=Q5
      IIN=I5
      PIN=P5
      ZIN=Z5
      x = x + (LIN / (QIN * (x + IIN) ** PIN + ZIN)) - (LIN / (QIN * (x - IIN) ** PIN + ZIN))
    return x

class E_layer_activation(nn.Module):
    def __init__(self):
        super().__init__() # init the base class
    def forward(self, input):
        return e_layer(input) # simply apply already implemented SiLU



def f_layer(input):
    x=input
    x[x<0]=0
    if DynamicLayer6:
      x = x + (L / (Q * (x + I) ** P + Z)) - (L / (Q * (x - I) ** P + Z))
    else:
      LIN=L6
      QIN=Q6
      IIN=I6
      PIN=P6
      ZIN=Z6
      x = x + (LIN / (QIN * (x + IIN) ** PIN + ZIN)) - (LIN / (QIN * (x - IIN) ** PIN + ZIN))
    return x

class F_layer_activation(nn.Module):
    def __init__(self):
        super().__init__() # init the base class
    def forward(self, input):
        return f_layer(input) # simply apply already implemented SiLU


def g_layer(input):
    x=input
    x[x<0]=0
    if DynamicLayer7:
      x = x + (L / (Q * (x + I) ** P + Z)) - (L / (Q * (x - I) ** P + Z))
    else:
      LIN=L7
      QIN=Q7
      IIN=I7
      PIN=P7
      ZIN=Z7
      x = x + (LIN / (QIN * (x + IIN) ** PIN + ZIN)) - (LIN / (QIN * (x - IIN) ** PIN + ZIN))
    return x

class G_layer_activation(nn.Module):
    def __init__(self):
        super().__init__() # init the base class
    def forward(self, input):
        return g_layer(input) # simply apply already implemented SiLU

def h_layer(input):
    x=input
    x[x<0]=0
    if DynamicLayer8:
      x = x + (L / (Q * (x + I) ** P + Z)) - (L / (Q * (x - I) ** P + Z))
    else:
      LIN=L8
      QIN=Q8
      IIN=I8
      PIN=P8
      ZIN=Z8
      x = x + (LIN / (QIN * (x + IIN) ** PIN + ZIN)) - (LIN / (QIN * (x - IIN) ** PIN + ZIN))    
    return x

class H_layer_activation(nn.Module):
    def __init__(self):
        super().__init__() # init the base class
    def forward(self, input):
        return h_layer(input) # simply apply already implemented SiLU

def setParams(L_,Q_,P_,Z_,I_):
    global L
    global Q
    global P
    global Z
    global I
    L = L_
    Q = Q_
    P = P_
    Z = Z_
    I = I_

    pass

activation_function = RELU()
print("activation_function set")

#..............................end of our part...................................#

In [ ]:
"""### Loading and Processing Dataset"""

# Download the dataset
dataset_url = 'https://s3.amazonaws.com/fast-ai-imageclas/cifar100.tgz'
download_url(dataset_url, '.')

#Extract from archive
with tarfile.open('./cifar100.tgz', 'r:gz') as tar:
  tar.extractall(path='./data')

# Look into the data directory
data_dir = './data/cifar100'
folders = os.listdir(data_dir + "/train")
classes=[]
for folder in folders:
  classes+=os.listdir(data_dir + "/train/"+folder)


#Data transforms (normalization and data augmentation)
stats = ((0.4914, 0.4822, 0.4465),
         (0.2023, 0.1994, 0.2010))

train_tfms = tt.Compose([tt.RandomCrop(32, padding=4, padding_mode='reflect'),
                         tt.RandomHorizontalFlip(),
                         tt.ToTensor(),
                         tt.Normalize(*stats, inplace=True)])

valid_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])

# PyTorch datasets
train_ds=[]
valid_ds=[]
for folder in folders:
  train_ds+=ImageFolder(data_dir+'/train/'+folder, train_tfms)
  valid_ds+=ImageFolder(data_dir+'/test/'+folder, valid_tfms)

if not setAll :
    data_S = list(range(0, len(train_ds),int(1/train_percent)))
    test_S = list(range(0, len(valid_ds), int(1/val_percent)))

    train_ds = torch.utils.data.Subset(train_ds, data_S)
    valid_ds = torch.utils.data.Subset(valid_ds, test_S)
    

# Pytorch Data Loaders
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size*2, num_workers=2, pin_memory=True)

"""### Uploading on GPU"""

### Using a GPU
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()
print(device)

train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)

# RESNET 9

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

"""Building our architecture:"""

def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              RELU()]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

def conv_block_dynamic(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              Dynamic()]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

def conv_block_A(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              A_layer_activation()]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

def conv_block_B(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              B_layer_activation()]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

def conv_block_C(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              C_layer_activation()]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

def conv_block_D(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              D_layer_activation()]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

def conv_block_E(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              E_layer_activation()]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

def conv_block_F(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              F_layer_activation()]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

def conv_block_G(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              G_layer_activation()]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

def conv_block_H(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              H_layer_activation()]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)        
class ResNet9_dynamic(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = conv_block_dynamic(in_channels, 64)
        self.conv2 = conv_block_dynamic(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block_dynamic(128, 128), conv_block_dynamic(128, 128))
        self.conv3 = conv_block_dynamic(128, 256, pool=True)
        self.conv4 = conv_block_dynamic(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block_dynamic(512, 512), conv_block_dynamic(512, 512))

        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                        nn.Flatten(),
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out
class ResNet9_layers(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = conv_block_A(in_channels, 64)
        self.conv2 = conv_block_B(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block_C(128, 128), conv_block_D(128, 128))
        self.conv3 = conv_block_E(128, 256, pool=True)
        self.conv4 = conv_block_F(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block_G(512, 512), conv_block_H(512, 512))

        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                        nn.Flatten(),
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

class ResNet9_relu(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))

        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                        nn.Flatten(),
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out


"""### Training the Model
The improvements in fit functions are:
1. Learning rate scheduling: Instead of using a fixed learning rate, we will use a learning rate scheduler, which will change the learning rate after every batch of training. We will use one cycle policy [1cycle policy](https://sgugger.github.io/the-1cycle-policy.html).
2. Weight Decay: A regularization technique which prevents the weights from becoming too large by adding an additional term to the loss function.
3. Gradient clipping: Apart from the layer weights and outputs, it also helpful to limit the values of gradients to a small range to prevent undesirable changes in parameters due to large gradient values

"""

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader,
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs,
                                                steps_per_epoch=len(train_loader))

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()

        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

# Evaluating the model before training

TRAIN

In [ ]:
# Evaluating the model before training

print("RELU on resnet9:")
activation_function=RELU()
combinedScores=0
bestRELU=0
for i in range(HowManyTimesReRunRelu):

  #model = to_device(ResNet9(3, 100), device)
  model = to_device(ResNet9_relu(3, 100), device)
  model
  history = [evaluate(model, valid_dl)]
  history

  # Commented out IPython magic to ensure Python compatibility.
  # %%time

  tic = time.time()
  history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl,
                                grad_clip=grad_clip,
                                weight_decay=weight_decay,
                                opt_func=opt_func)
  toc = time.time()
  RELUscore=history[len(history)-1]["val_acc"]
  if addMaxRelu:
    if RELUscore>bestRELU:
      bestRELU=RELUscore

  if addMean:
    combinedScores+=RELUscore
  print("time took:", toc - tic,"relu score is:",RELUscore)
if addMaxRelu:
    print("RELU best score:",bestRELU)
    RELUscore=bestRELU

if addMean:  
  print("mean:",combinedScores/HowManyTimesReRunRelu)



if checkCertainParams:
  print("founded function on resnet9:")
  maxCertainParams=0
  minCertainParams=1
  combinedScoresCertainParams=0
  for i in range(howManyTimeToCheckTheCertainParams):
    activation_function=Dynamic()
    model = to_device(ResNet9_layers(3, 100), device)
    model
    history = [evaluate(model, valid_dl)]
    history

    tic = time.time()
    history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl,
                                grad_clip=grad_clip,
                                weight_decay=weight_decay,
                                opt_func=opt_func)
    toc = time.time()
    score_cetrain=history[len(history)-1]["val_acc"]
    if addMaxToCertain:
      if score_cetrain> maxCertainParams:
        maxCertainParams=score_cetrain
      pass
    if addMinToCertain:
      if score_cetrain< minCertainParams:
        minCertainParams=score_cetrain
      pass
    if addMeanToCertain:   
      combinedScoresCertainParams+=score_cetrain
      pass 
    print("time took:", toc - tic)
  if addMaxToCertain:
    print("max score of the certain params is:",maxCertainParams )
    pass 
  if addMinToCertain:
    print("min score of the certain params is:",minCertainParams )
    pass     
  if addMeanToCertain:
    if not howManyTimeToCheckTheCertainParams==0:
      print("mean score of the certain params is:",combinedScoresCertainParams/howManyTimeToCheckTheCertainParams)
    pass        

else:
  bestScore=0
  bestI=0
  bestQ=0
  bestL=0
  f = open("result.txt", "a")

  #Dynamic
  activation_function = Dynamic()
  I_=0
  Q_=0
  L_=0
  for i in range(SIZEI):
      for l in range(SIZEL):
          for q in range(SIZEQ):
              print(100*(q+l*SIZEQ+i*SIZEQ*SIZEL)/(SIZEI*SIZEL*SIZEQ),"percent done")
              model = to_device(ResNet9_layers(3, 100), device)
              to_device(model, device)
              history = [evaluate(model, valid_dl)]

              I_=fromI+i*(toI-fromI)/SIZEI
              Q_=fromQ+q*(toQ-fromQ)/SIZEQ
              L_=fromL+l*(toL-fromL)/SIZEL

              if(LogarithmINC):
                  I_=(((toI/fromI)**(1/SIZEI))**i)*fromI
                  Q_=(((toQ/fromQ)**(1/SIZEQ))**q)*fromQ
                  L_=(((toL/fromL)**(1/SIZEL))**l)*fromL
                  pass
              setParams(L_,Q_,P,Z,I_)
              tic = time.time()
              history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl,
                                          grad_clip=grad_clip,
                                          weight_decay=weight_decay,
                                          opt_func=opt_func)
              toc = time.time()
              print("time took:", toc - tic)
              score=history[len(history)-1]["val_acc"]
              f.write("score:"+str(score)+","+"i:"+str(I)+","+"q:"+str(Q)+","+"l:"+str(L)+","+"z:"+str(Z)+","+"P:"+str(P)+","+"val_percent:"+str(val_percent)+","+
                "train_percent:"+str(train_percent)+","+
                "batch:"+str(batch_size)+","+"epochs:"+str(epochs)+","+
                "lr:"+str(lr)+","+"max_lr:"+str(max_lr)+","+
                "grad_clip:"+str(grad_clip)+","+
                "weight_decay:"+str(weight_decay)+","+str(history)+"\n")
              if score>bestScore:
                  bestI = I_
                  bestQ = Q_
                  bestL = L_
                  bestScore=score
              print(score,"with i=",I_,"q=",Q_,"l=",L_)


  f.close()
  print("best score:",bestScore,"with i=",bestI,"q=",bestQ,"l=",bestL,"relu best score was:",RELUscore)